In [13]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [14]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() !=api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [15]:
openai=OpenAI()
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I’m glad you reached out. How can I assist you today?


In [21]:
headers={
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text=soup.body.get_text(separator="\n", strip=True)

In [24]:
ant = Website("https://www.anthropic.com/")
print(ant.title)
print(ant.text)

Home \ Anthropic
Skip to main content
Skip to footer
Claude
Product
Claude
Claude Code
Plans
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s Character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Code modernization
Coding
Customer support
Education
Financial services
Government
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineering at Anthropic
Anthropic Academy
Company
About
Careers
Events
Engineering
Building effective agents
News
EN
This is some text inside of a div block.
Try Claude
Try Claude
Try Claude
Down

In [25]:
system_prompt="You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [26]:
def user_prompt_for(website):
    user_prompt=f"You are looking at a website titled {website.title}"
    user_prompt+="\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt+=website.text
    return user_prompt

In [27]:
print(user_prompt_for(ant))

You are looking at a website titled Home \ Anthropic
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Skip to footer
Claude
Product
Claude
Claude Code
Plans
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s Character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Code modernization
Coding
Customer support
Education
Financial services
Government
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer

In [30]:
def messages_for(website):
    return [
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt_for(website)}
    ]

In [31]:
messages_for(ant)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Home \\ Anthropic\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to main content\nSkip to footer\nClaude\nProduct\nClaude\nClaude Code\nPlans\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s Character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCode modernization\nCoding\nCustomer support\nEducation\nFinancial services\nGovernment\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\

In [38]:
def summarize(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [39]:
summarize("https://www.anthropic.com/")

'# Anthropic Website Summary\n\nThe Anthropic website presents information primarily about its AI products, focusing particularly on the Claude AI model family. Key highlights include:\n\n## Products\n- **Claude Opus 4.1**: The latest and most intelligent AI model now available.\n- **Claude Sonnet 4**: Another powerful model that facilitates advanced coding and AI applications.\n- **Claude Code**: A tool designed for building AI-powered applications and enhancing custom experiences. \n\n## Plans and Pricing\nVarious plans are offered:\n- **Max plan**\n- **Team plan**\n- **Enterprise plan**\n\n## Research and Safety\nAnthropic emphasizes AI safety and responsible development. It shares insights on:\n- **AI Safety**: Core beliefs regarding the safe deployment of AI technologies.\n- **Responsible Scaling Policy**: A commitment to sustainable and ethical AI practices.\n\n## Learning Resources\nThe site features the **Anthropic Academy**, which provides resources for learning to build with 

In [43]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [44]:
display_summary("https://www.anthropic.com/")

# Summary of Anthropic Website

The Anthropic website is centered around their AI products, primarily featuring **Claude**, an intelligent AI model designed for a variety of applications, including coding and customer support. Key highlights include:

- **Claude Models**: The latest models include **Claude Opus 4.1** and **Claude Sonnet 4**, both noted for their capabilities in coding and AI agents.
- **API and Development**: Users are encouraged to build custom applications using Claude through detailed documentation and resources available on their site.
- **Core Principles**: Anthropic emphasizes AI safety and responsible development, incorporating a policy of transparency and ethical considerations in AI scaling. They outline their commitment to creating AI technologies that prioritize human benefit.

## Recent Announcements
- **ISO 42001 Certification**: This certification reflects a commitment to high standards in managing AI technologies.
- **Release of Claude Opus 4.1**: Announced as the most advanced AI model to date, ready to handle extensive tasks effectively.

The website also features learning resources, customer stories, and professional development opportunities through the **Anthropic Academy**.